## 1. Load the dataset
We will combine the  Description and Patient text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [196]:
# imports
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import time

In [197]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [198]:
# load & inspect dataset
df = pd.read_csv("../2-Data/dialogues.csv", sep = '\t')

In [199]:
df = df.dropna().head(1000)

In [200]:
df

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...
...,...,...,...
995,Q. My lax les is 38 cm with inflamed gastric f...,"Hello doctor, My lax les is 38 cm with inflame...",Hello. Gastritis is an inflammation of stomach...
996,Q. I am suffering from mood swings. Kindly adv...,"Hello doctor,I want to get some information re...",Hello. Let me answer your questions via some b...
997,Q. I am having swollen lymph node in my neck. ...,"Hello doctor, I went to the chiropractor and g...",Hello. I do not think that because of chiropra...
998,Q. How good is Albenza for a raccoon roundworm...,"Hello doctor,I am concerned about a possible r...",Hello. Albendazole 400 mg single star dose is ...


In [201]:
df["combined"] = (
    "Description: " + df.Description.str.strip() + "; Patient: " + df.Patient.str.strip()+ "; Doctor: " + df.Doctor.str.strip()
)
df.head(2)

,Description,Patient,Doctor,combined
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Description: Q. What does abutment of the nerv...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Description: Q. What should I do to reduce my ...


In [202]:
#df["combined"] = ( "Description: " + df.Description.str.strip() + "; Patient: " + df.Patient.str.strip())
#df.head(2)

In [203]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out

In [204]:
df

,Description,Patient,Doctor,combined
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Description: Q. What does abutment of the nerv...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Description: Q. What should I do to reduce my ...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...,Description: Q. I have started to get lots of ...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...,Description: Q. Why do I have uncomfortable fe...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...,Description: Q. My symptoms after intercourse ...
...,...,...,...,...
995,Q. My lax les is 38 cm with inflamed gastric f...,"Hello doctor, My lax les is 38 cm with inflame...",Hello. Gastritis is an inflammation of stomach...,Description: Q. My lax les is 38 cm with infla...
996,Q. I am suffering from mood swings. Kindly adv...,"Hello doctor,I want to get some information re...",Hello. Let me answer your questions via some b...,Description: Q. I am suffering from mood swing...
997,Q. I am having swollen lymph node in my neck. ...,"Hello doctor, I went to the chiropractor and g...",Hello. I do not think that because of chiropra...,Description: Q. I am having swollen lymph node...
998,Q. How good is Albenza for a raccoon roundworm...,"Hello doctor,I am concerned about a possible r...",Hello. Albendazole 400 mg single star dose is ...,Description: Q. How good is Albenza for a racc...


In [205]:
encoding = tiktoken.get_encoding(embedding_encoding)
# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

C:\Users\rusla\AppData\Local\Temp\ipykernel_26316\751895340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))


1000

In [206]:
df

,Description,Patient,Doctor,combined,n_tokens
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Description: Q. What does abutment of the nerv...,95
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Description: Q. What should I do to reduce my ...,519
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...,Description: Q. I have started to get lots of ...,285
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...,Description: Q. Why do I have uncomfortable fe...,324
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...,Description: Q. My symptoms after intercourse ...,442
...,...,...,...,...,...
995,Q. My lax les is 38 cm with inflamed gastric f...,"Hello doctor, My lax les is 38 cm with inflame...",Hello. Gastritis is an inflammation of stomach...,Description: Q. My lax les is 38 cm with infla...,214
996,Q. I am suffering from mood swings. Kindly adv...,"Hello doctor,I want to get some information re...",Hello. Let me answer your questions via some b...,Description: Q. I am suffering from mood swing...,491
997,Q. I am having swollen lymph node in my neck. ...,"Hello doctor, I went to the chiropractor and g...",Hello. I do not think that because of chiropra...,Description: Q. I am having swollen lymph node...,395
998,Q. How good is Albenza for a raccoon roundworm...,"Hello doctor,I am concerned about a possible r...",Hello. Albendazole 400 mg single star dose is ...,Description: Q. How good is Albenza for a racc...,240


There are different ways to convert  text into a vector or into embeddings.

Unfortunately, most good methods to get embeddings in Python are not free.


## 2. Get embeddings using  SentenceTransformers

Let us use SentenceTransformers, a Python framework for state-of-the-art sentence, text, and image embeddings. The initial work is described in our paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks.

First we verify that Torch is CUDA capable

In [207]:
import torch
torch.cuda.is_available()

True

We define our list of sentences. You can use a larger list (it is best to use a list of sentences for easier processing of each sentence)

We can install Sentence BERT using:
`!pip install sentence-transformers`



Step 1: We will then load the pre-trained BERT model. There are many other pre-trained models available.

In [208]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

We proceed to test the embeding creation

In [209]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']
#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [210]:
sentence

['This framework generates embeddings for each input sentence']

In [211]:
def get_embeddings(x,transformer='paraphrase-MiniLM-L6-v2'):
    model = SentenceTransformer(transformer)
    #Sentences we want to encode
    sentence =x
    #Sentences are encoded by calling model.encode()
    embedding = model.encode(sentence)
    return embedding

In [212]:
# This may take a few minutes
embedding_mod='paraphrase-MiniLM-L6-v2'
#df["embedding"] = df.combined.apply(lambda x: get_embeddings(x, transformer=embedding_mod))

In [213]:
#embedding_doctor
# This may take a few minutes
df["embedding"] = df.Doctor.apply(lambda x: get_embeddings(x, transformer=embedding_mod))

In [214]:
df

,Description,Patient,Doctor,combined,n_tokens,embedding
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...,Description: Q. What does abutment of the nerv...,95,"[-0.109211065, -0.17469415, 0.18996556, 0.0599..."
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...,Description: Q. What should I do to reduce my ...,519,"[-0.014065318, 0.0440334, 0.26095688, 0.086799..."
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...,Description: Q. I have started to get lots of ...,285,"[-0.39175138, -0.025890486, -0.024644196, -0.0..."
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...,Description: Q. Why do I have uncomfortable fe...,324,"[-0.29406005, -0.31878802, 0.27588362, 0.09649..."
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...,Description: Q. My symptoms after intercourse ...,442,"[-0.36187398, 0.18491694, -0.3090741, -0.30197..."
...,...,...,...,...,...,...
995,Q. My lax les is 38 cm with inflamed gastric f...,"Hello doctor, My lax les is 38 cm with inflame...",Hello. Gastritis is an inflammation of stomach...,Description: Q. My lax les is 38 cm with infla...,214,"[-0.1555396, -0.44157797, -0.15364785, 0.25760..."
996,Q. I am suffering from mood swings. Kindly adv...,"Hello doctor,I want to get some information re...",Hello. Let me answer your questions via some b...,Description: Q. I am suffering from mood swing...,491,"[-0.2296337, 0.119730674, 0.37153018, 0.062901..."
997,Q. I am having swollen lymph node in my neck. ...,"Hello doctor, I went to the chiropractor and g...",Hello. I do not think that because of chiropra...,Description: Q. I am having swollen lymph node...,395,"[-0.10149522, -0.33532476, 0.40812746, -0.2713..."
998,Q. How good is Albenza for a raccoon roundworm...,"Hello doctor,I am concerned about a possible r...",Hello. Albendazole 400 mg single star dose is ...,Description: Q. How good is Albenza for a racc...,240,"[-0.06408733, 0.17669381, 0.09132431, -0.09456..."


In [215]:
from ast import literal_eval
import numpy as np

In [216]:
df["embedding"] = df.embedding.apply(np.array)  # convert string to numpy array


In [217]:
#df["embedding_doctor"] = df.embedding_doctor.apply(np.array)  # convert string to numpy array


In [218]:
df.to_pickle("../2-Data/dialogues_embededd.pkl")

In [195]:
#df.to_csv("../2-Data/dialogues_embededd.csv", sep = '\t', encoding='utf-8', index=False)

## 3. Get embeddings using OpenAI (optional)
If we have a subscription in OpenAI, you can follow the following steps.
Is optional, we are going to use the previous method.

In [24]:
# Python program to read
# json file
import json
# Opening JSON file
f = open('./credentials/api.json')
# returns JSON object as
# a dictionary
data = json.load(f)

In [28]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage
import openai
openai.api_key = data['OPENAI_API_KEY']
# Closing file
f.close()

In [42]:
# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [ ]:
df.to_csv("../2-Data/dialogues_embededd_openai.csv", sep='\t', encoding='utf-8', index=False)

## Additional Notes (not neeeded)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = ["I am doga.",
       "I am a dog"]
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
# list of text documents
text = ["I am  doc.", "I am dog"]
# create the transform
vectorizer = HashingVectorizer(n_features=20)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())